# $ \LaTeX $  **で入力できる電卓**

## **概要**

$ \LaTeX $ 形式で入出力できる関数電卓です。

2つの動作モードを持ち、計算機モードでは数式の計算が、 \\
ソルバモードでは方程式を解くことができます。

$ \LaTeX $ に詳しくない方でも簡単に扱えるように、基本的な数学記号を入力できるボタンを備えています。

## **操作方法**

0. ライブラリのインストールのために上部のセルを実行してください。

1. 下部のセルを実行してください。

2. 出力最上部のModeボタンで計算機モードと方程式ソルバモードを切り替えてください。

3. Inputテキストエリアに $ \LaTeX $ 形式で計算式もしくは方程式を入力してください。 \\
  必要に応じてMathematical Symbolsセクションのボタンを使用してください。

4. (方程式ソルバの場合) \\
  ControlsセクションのVariableテキストエリアに解く文字を入力してください。

5. Executeボタンを押下してください。

### **注意点**

このプログラムは`sympy 1.14.0`に依存しているので必ず上部のセルを実行してインストールしてください。

その際、依存関係を解決するために`torch`をアンインストールします。

時間がかかるので、アンインストールが完了するまでしばらくお待ちください。

## **制作者のコメント**

なるべくクラス化を行い、可読性を高めました。

また、NumPy式のDocstringを付与して関数やクラスを保守しやすくしました。

しかし、ifを多用、ネストしてしまった点は保守性を下げている要因だと感じました。
リファクタリングする際はifのネストを避けられるようにすればより可読性、保守性が高まると思います。

ほかに、事前調査とライブラリの理解に時間を割きすぎて設計を詰められなかった点、outputを直書きしてしまった点も反省すべき点だと思います。

## **作成時間**

作成時間：**約25時間**


In [1]:
%reset -f

%pip uninstall omegaconf torch -y
%pip install antlr4-python3-runtime==4.11
%pip install -U sympy

In [2]:
%reset -f

from sympy.parsing.latex import parse_latex as l2s
import sympy as sp
import ipywidgets as widgets
from enum import Enum
from IPython.display import display, Math, clear_output
clear_output()


class Mode(Enum):
    """
    Enumeration for operation modes of the mathematical interface.

    Attributes
    ----------
    CALCULATOR : str
        Mode for evaluating mathematical expressions.
    SOLVER : str
        Mode for solving equations.
    """
    CALCULATOR = "Calculator"
    SOLVER = "Solver"


output = widgets.Output(
    layout=widgets.Layout(width='95%', height='200px', border='1px solid gray')
)

def create_symbol_button(text_area: widgets.Textarea, desc: str):
    """
    Create a button that inserts mathematical symbols into a text area.

    Parameters
    ----------
    text_area : widgets.Textarea
        The textarea widget where symbols will be inserted.
    desc : str
        The LaTeX symbol or command to be inserted.

    Returns
    -------
    widgets.Button
        A button widget that inserts the symbol when clicked.
    """
    button = widgets.Button(
        description=desc,
        layout=widgets.Layout(width='100px', height='30px'),
        style={'font_size': '10px'}
    )

    def on_click(b):
        current_value = text_area.value
        text_area.value = current_value + " " + desc + " "

    button.on_click(on_click)
    return button

def init_interface():
    """
    Initialize the user interface components for the mathematical calculator.

    Creates and displays all UI widgets including input areas, buttons,
    and symbol palette for mathematical expressions.

    Notes
    -----
    This function creates global variables for the interface components
    and displays them using IPython widgets.
    """
    global input_area, variable_input, mode, output, eval_button, clear_button

    input_area = widgets.Textarea(
        value=r"\lim_{x \to 0} \frac{\sin x}{x}",
        placeholder="LaTeX expression or equation",
        description="Input:",
        layout=widgets.Layout(width='95%', height='80px')
    )

    variable_input = widgets.Text(
        value="x",
        placeholder="Variable to solve for",
        description="Variable:",
        layout=widgets.Layout(width='200px')
    )

    mode = widgets.ToggleButtons(
        options=[Mode.CALCULATOR.value, Mode.SOLVER.value],
        description="Mode:",
        value=Mode.CALCULATOR.value,
        style={'button_width': '100px'}
    )

    eval_button = widgets.Button(
        description="Execute",
        button_style='success',
        layout=widgets.Layout(width='100px')
    )

    clear_button = widgets.Button(
        description="Clear",
        button_style='warning',
        layout=widgets.Layout(width='100px')
    )

    # Create symbol buttons
    sqrt_btn = create_symbol_button(input_area, r"\sqrt{}")
    frac_btn = create_symbol_button(input_area, r"\frac{}{}")
    power_btn = create_symbol_button(input_area, r"^{}")
    sum_btn = create_symbol_button(input_area, r"\sum_{i=1}^{n}")
    prod_btn = create_symbol_button(input_area, r"\prod_{i=1}^{n}")

    limit_btn = create_symbol_button(input_area, r"\lim_{x \to a}")
    diff_btn = create_symbol_button(input_area, r"\frac{d}{dx}")
    integral_btn = create_symbol_button(input_area, r"\int")

    sin_btn = create_symbol_button(input_area, r"\sin")
    cos_btn = create_symbol_button(input_area, r"\cos")
    tan_btn = create_symbol_button(input_area, r"\tan")

    log_btn = create_symbol_button(input_area, r"\log")
    ln_btn = create_symbol_button(input_area, r"\ln")
    exp_btn = create_symbol_button(input_area, r"e^{}")

    alpha_btn = create_symbol_button(input_area, r"\alpha")
    beta_btn = create_symbol_button(input_area, r"\beta")
    gamma_btn = create_symbol_button(input_area, r"\gamma")
    delta_btn = create_symbol_button(input_area, r"\delta")
    pi_btn = create_symbol_button(input_area, r"\pi")
    theta_btn = create_symbol_button(input_area, r"\theta")

    infty_btn = create_symbol_button(input_area, r"\infty")
    pm_btn = create_symbol_button(input_area, r"\pm")

    # Arrange buttons in grid
    buttons = widgets.GridspecLayout(5, 6,
                                    layout=widgets.Layout(width='95%'))

    buttons[0, 0] = sqrt_btn
    buttons[0, 1] = frac_btn
    buttons[0, 2] = power_btn
    buttons[0, 3] = sum_btn
    buttons[0, 4] = prod_btn
    buttons[0, 5] = infty_btn

    buttons[1, 0] = limit_btn
    buttons[1, 1] = diff_btn
    buttons[1, 2] = integral_btn
    buttons[1, 3] = pm_btn

    buttons[2, 0] = sin_btn
    buttons[2, 1] = cos_btn
    buttons[2, 2] = tan_btn

    buttons[3, 0] = log_btn
    buttons[3, 1] = ln_btn
    buttons[3, 2] = exp_btn

    buttons[4, 0] = alpha_btn
    buttons[4, 1] = beta_btn
    buttons[4, 2] = gamma_btn
    buttons[4, 3] = delta_btn
    buttons[4, 4] = pi_btn
    buttons[4, 5] = theta_btn

    control_box = widgets.HBox([eval_button, clear_button, variable_input])

    display(
        mode,
        widgets.HTML("<h4>Mathematical Symbols:</h4>"),
        buttons,
        widgets.HTML("<h4>Input:</h4>"),
        input_area,
        widgets.HTML("<h4>Controls:</h4>"),
        control_box,
        widgets.HTML("<h4>Output:</h4>"),
        output
    )

class Base:
    """
    Base class for mathematical expression processing.

    This class provides common functionality for parsing LaTeX expressions
    and serves as a parent class for Calculator and Solver.

    Parameters
    ----------
    user_input : str
        LaTeX string representing a mathematical expression or equation.

    Attributes
    ----------
    user_input : str
        The original LaTeX input string.
    expr : sympy.Basic or None
        The parsed SymPy expression.
    result : sympy.Basic or None
        The result of the operation performed on the expression.
    """

    def __init__(self, user_input: str):
        self.user_input = user_input
        self.expr = None
        self.result = None

    def parse(self):
        """
        Parse the LaTeX input string into a SymPy expression.

        Returns
        -------
        Base
            Returns self for method chaining.

        Raises
        ------
        ValueError
            If the LaTeX expression cannot be parsed.
        """
        try:
            self.expr = l2s(self.user_input)
        except Exception as e:
            print(f"Error while parsing LaTeX expression.\nPlease restart session.\n{e}")
            raise ValueError("Error while parsing LaTeX expression.")
        return self


class Calculator(Base):
    """
    Calculator class for evaluating mathematical expressions.

    This class extends Base to provide functionality for simplifying
    and evaluating mathematical expressions.

    Parameters
    ----------
    user_input : str
        LaTeX string representing a mathematical expression.
    """

    def __init__(self, user_input: str):
        super().__init__(user_input)

    def evaluate(self):
        """
        Evaluate or simplify the parsed mathematical expression.

        Returns
        -------
        Calculator
            Returns self for method chaining.

        Notes
        -----
        Different types of expressions are handled differently:
        - Matrix expressions are evaluated using doit()
        - Basic SymPy expressions are simplified
        - Other types are returned as-is
        """
        match self.expr:
            case self.expr if isinstance(self.expr, sp.MatrixExpr):
                self.result = self.expr.doit()
            case self.expr if isinstance(self.expr, sp.Basic):
                self.result = sp.simplify(self.expr)
            case _:
                self.result = self.expr
        return self

    def display_result(self):
        """
        Display the calculation result in mathematical notation.

        Shows both the original expression and the result using LaTeX
        formatting for SymPy expressions, or plain text for others.
        """
        if isinstance(self.result, sp.Basic):
            display(Math(f"{sp.latex(self.expr)} = {sp.latex(self.result)}"))
        else:
            print(self.result)


class Solver(Base):
    """
    Solver class for solving mathematical equations.

    This class extends Base to provide functionality for solving
    equations with respect to specified variables.

    Parameters
    ----------
    user_input : str
        LaTeX string representing a mathematical equation.
    variables : str, sympy.Symbol, or None, optional
        The variable(s) to solve for. If None, will be auto-detected.

    Attributes
    ----------
    variables : sympy.Symbol
        The variable to solve for.
    """

    def __init__(self, user_input, variables=None):
        super().__init__(user_input)
        self.variables = variables

    def solve(self):
        """
        Solve the mathematical equation for the specified variable.

        Returns
        -------
        Solver
            Returns self for method chaining.

        Raises
        ------
        ValueError
            If no variables are found in the expression or if the
            equation cannot be solved.

        Notes
        -----
        If no variable is specified, the method will:
        1. Use the single free symbol if there's only one
        2. Prefer 'x' if it exists among multiple free symbols
        3. Use the first free symbol as fallback
        """
        try:
            if self.variables is None:
                free_symbols = self.expr.free_symbols
                if len(free_symbols) == 1:
                    self.variables = list(free_symbols)[0]
                elif len(free_symbols) > 1:
                    if sp.Symbol("x") in free_symbols:
                        self.variables = sp.Symbol("x")
                    else:
                        self.variables = list(free_symbols)[0]
                else:
                    raise ValueError("No variables found in the expression")
            else:
                if isinstance(self.variables, str):
                    self.variables = sp.Symbol(self.variables)

            self.result = sp.solve(self.expr, self.variables)
            return self

        except Exception as e:
            print(f"Error while solving equation: {e}")
            raise ValueError("Could not solve the equation")

    def display_result(self):
        """
        Display the equation and its solution(s) in mathematical notation.

        Shows the original equation being solved and the solution(s)
        using LaTeX formatting. Handles single solutions, multiple
        solutions, and cases with no real solutions.
        """
        if isinstance(self.expr, sp.Eq):
            expr_latex = sp.latex(self.expr)
        else:
            expr_latex = f"{sp.latex(self.expr)} = 0"

        display(Math(f"\\text{{Solving: }} {expr_latex}"))

        if self.result:
            if len(self.result) == 1:
                display(Math(f"{sp.latex(self.variables)} = {sp.latex(self.result[0])}"))
            else:
                result = ", ".join([sp.latex(sol) for sol in self.result])
                display(Math(f"{sp.latex(self.variables)} = \\left\\{{{result}\\right\\}}"))
        else:
            display(Math(r"\text{No real solutions found}"))


def setup_handlers():
    """
    Set up event handlers for the user interface components.

    Configures click handlers for buttons and change observers for
    mode selection. Handles evaluation, clearing, and mode switching.

    Notes
    -----
    This function defines nested event handler functions and binds
    them to the appropriate widget events.
    """
    def on_eval_clicked(b):
      with output:
        clear_output()

        user_input = input_area.value.strip()
        current_mode = mode.value

        if not user_input:
          print("Please enter an expression.")
          return

        try:
          if current_mode == Mode.CALCULATOR.value:
            calculator = Calculator(user_input)
            calculator.parse().evaluate().display_result()

          else:  # Solver mode
            variable_name = variable_input.value.strip() if variable_input.value.strip() else 'x'
            solver = Solver(user_input, variable_name)
            solver.parse().solve().display_result()

        except ValueError as e:
          print(f"❌ Error: {e}")
        except Exception as e:
          print(f"❌ Unexpected error: {e}")

    @output.capture()
    def on_clear_clicked(b):
        """Handle clear button click."""
        input_area.value = ""
        clear_output()

    @output.capture()
    def on_mode_changed(change):
        """Handle mode selection change."""
        if change['new'] == Mode.CALCULATOR.value:
            input_area.placeholder = "LaTeX expression (e.g., x^2 + 2*x + 1)"
            input_area.value = r"\lim_{x \to 0} \frac{\sin x}{x}"
        else:
            input_area.placeholder = "LaTeX equation (e.g., x^2 - 4 = 0)"
            input_area.value = "x^2 - 4 = 0"

        clear_output()

    eval_button.on_click(on_eval_clicked)
    clear_button.on_click(on_clear_clicked)
    mode.observe(on_mode_changed, names='value')

def main():
  """
  Main function to initialize and run the mathematical calculator interface.

  Initializes the user interface and sets up event handlers.
  This is the entry point for the application.
  """
  init_interface()
  setup_handlers()

if __name__ == "__main__":
  main()

ToggleButtons(description='Mode:', options=('Calculator', 'Solver'), style=ToggleButtonsStyle(button_width='10…

HTML(value='<h4>Mathematical Symbols:</h4>')

GridspecLayout(children=(Button(description='\\sqrt{}', layout=Layout(grid_area='widget001', height='30px', wi…

HTML(value='<h4>Input:</h4>')

Textarea(value='\\lim_{x \\to 0} \\frac{\\sin x}{x}', description='Input:', layout=Layout(height='80px', width…

HTML(value='<h4>Controls:</h4>')

HTML(value='<h4>Output:</h4>')

Output(layout=Layout(border='1px solid gray', height='200px', width='95%'))